In [2]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,452 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,172 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,323 kB]
Get:13 http://archive.ub

In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [5]:
# Resolve paths to the file added through Spark
spark.sparkContext.addFile(url)
home_sales_data = spark.read.option('header', 'true').csv(SparkFiles.get("home_sales_revised.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")

# Show DataFrame
home_sales_data.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [6]:
# 2. Create a temporary view of the DataFrame.
home_sales_data.createOrReplaceTempView('home_sales')

In [7]:
# View the data using SparkSQL
spark.sql("select * from home_sales limit 10").show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [12]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
spark.sql("""
  SELECT INT(date_built) AS build_year,
    ROUND(AVG(price),2) AS average_price
  From home_sales
  WHERE bedrooms = 4
  Group By build_year
  Order By build_year
""").show()


+----------+-------------+
|build_year|average_price|
+----------+-------------+
|      2010|    296800.75|
|      2011|     302141.9|
|      2012|    298233.42|
|      2013|    299999.39|
|      2014|    299073.89|
|      2015|    307908.86|
|      2016|    296050.24|
|      2017|    296576.69|
+----------+-------------+



In [13]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
spark.sql("""
  SELECT INT(date_built) AS build_year,
    ROUND(AVG(price),2) AS average_price
  From home_sales
  WHERE bedrooms = 3 AND bathrooms =3
  Group By build_year
  Order By build_year
""").show()



+----------+-------------+
|build_year|average_price|
+----------+-------------+
|      2010|    292859.62|
|      2011|    291117.47|
|      2012|    293683.19|
|      2013|    295962.27|
|      2014|    290852.27|
|      2015|     288770.3|
|      2016|    290555.07|
|      2017|    292676.79|
+----------+-------------+



In [14]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
spark.sql("""
  SELECT INT(date_built) AS build_year,
    ROUND(AVG(price),2) AS average_price
  From home_sales
  WHERE bedrooms = 3 AND bathrooms =3 AND floors = 2 AND sqft_living >= 2000
  Group By build_year
  Order By build_year
""").show()


+----------+-------------+
|build_year|average_price|
+----------+-------------+
|      2010|    285010.22|
|      2011|    276553.81|
|      2012|    307539.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2015|    297609.97|
|      2016|     293965.1|
|      2017|    280317.58|
+----------+-------------+



In [15]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.
spark.sql("""
  SELECT INT(view) AS views,
    ROUND(AVG(price),2) AS average_price
  From home_sales
  Group By views
  HAVING AVG(price) >= 350000
  Order By views desc
""").show()


start_time = time.time()

print("--- %s seconds ---" % (time.time() - start_time))

+-----+-------------+
|views|average_price|
+-----+-------------+
|  100|    1026669.5|
|   99|   1061201.42|
|   98|   1053739.33|
|   97|   1129040.15|
|   96|   1017815.92|
|   95|    1054325.6|
|   94|    1033536.2|
|   93|   1026006.06|
|   92|    970402.55|
|   91|   1137372.73|
|   90|   1062654.16|
|   89|   1107839.15|
|   88|   1031719.35|
|   87|    1072285.2|
|   86|   1070444.25|
|   85|   1056336.74|
|   84|   1117233.13|
|   83|   1033965.93|
|   82|    1063498.0|
|   81|   1053472.79|
+-----+-------------+
only showing top 20 rows

--- 0.00018548965454101562 seconds ---


In [16]:
# 7. Cache the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [17]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [18]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.
spark.sql("""
  SELECT INT(view) AS views,
    ROUND(AVG(price),2) AS average_price
  From home_sales
  Group By views
  HAVING AVG(price) >= 350000
  Order By views desc
""").show()

start_time = time.time()

print("--- %s seconds ---" % (time.time() - start_time))


+-----+-------------+
|views|average_price|
+-----+-------------+
|  100|    1026669.5|
|   99|   1061201.42|
|   98|   1053739.33|
|   97|   1129040.15|
|   96|   1017815.92|
|   95|    1054325.6|
|   94|    1033536.2|
|   93|   1026006.06|
|   92|    970402.55|
|   91|   1137372.73|
|   90|   1062654.16|
|   89|   1107839.15|
|   88|   1031719.35|
|   87|    1072285.2|
|   86|   1070444.25|
|   85|   1056336.74|
|   84|   1117233.13|
|   83|   1033965.93|
|   82|    1063498.0|
|   81|   1053472.79|
+-----+-------------+
only showing top 20 rows

--- 9.226799011230469e-05 seconds ---


In [19]:
spark.sql("uncache table home_sales")


DataFrame[]

In [20]:
spark.catalog.isCached('home_sales')


False

In [21]:
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
home_sales_parquet = spark.read.option('header', 'true').csv(SparkFiles.get("home_sales_revised.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")


In [22]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales_parquet.write.mode("overwrite").parquet("home_sales_parquet")

df = spark.read.parquet("home_sales_parquet")

df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned")

In [23]:
# 11. Read the parquet formatted data.
df = spark.read.parquet("home_sales_parquet")


In [24]:
# 12. Create a temporary table for the parquet data.
df.createOrReplaceTempView('temp_home_sales')


In [25]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.
spark.sql("""
  SELECT INT(view) AS views,
    ROUND(AVG(price),2) AS average_price
  From temp_home_sales
  Group By views
  HAVING AVG(price) >= 350000
  Order By views desc
""").show()

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

+-----+-------------+
|views|average_price|
+-----+-------------+
|  100|    1026669.5|
|   99|   1061201.42|
|   98|   1053739.33|
|   97|   1129040.15|
|   96|   1017815.92|
|   95|    1054325.6|
|   94|    1033536.2|
|   93|   1026006.06|
|   92|    970402.55|
|   91|   1137372.73|
|   90|   1062654.16|
|   89|   1107839.15|
|   88|   1031719.35|
|   87|    1072285.2|
|   86|   1070444.25|
|   85|   1056336.74|
|   84|   1117233.13|
|   83|   1033965.93|
|   82|    1063498.0|
|   81|   1053472.79|
+-----+-------------+
only showing top 20 rows

--- 0.00013446807861328125 seconds ---


In [26]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table temp_home_sales")


DataFrame[]

In [27]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('temp_home_sales')


False